In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
#import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/output_eur_usd_trimmed.csv")

In [ ]:

df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.16
3538    2022.03.17
3539    2022.03.18
3540    2022.03.20
3541    2022.03.21
Name: Date, Length: 3542, dtype: object

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
scaler = scaler.fit(df_numpy[:3000])
df_scalled = scaler.transform(df_numpy)
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
df_train = df_scalled[0:2500]
df_val = df_scalled[2500:3000] #300
df_game = df_scalled[3001:]

In [ ]:
lookback = 30 #15
step = 1
delay = 30 #0
batch_size = 128

In [ ]:
float_data = np.array(df_scalled).astype('float32')
float_train_data = np.array(df_train).astype('float32')
float_val_data = np.array(df_val).astype('float32')
float_game_data = np.array(df_game).astype('float32')

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
train_gen = generator(float_train_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=2499 -delay,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_val_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=500-delay ,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_game_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=540 -delay,
step=step,
batch_size=batch_size)

In [ ]:
train_steps = (2500 - lookback)
val_steps = (500 - lookback)
test_steps = (541 - lookback)

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(120,
  activation='sigmoid',
  input_shape=(None, float_train_data.shape[-1])))
model.add(layers.Dense(160,
  activation='sigmoid',))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=100,
  epochs=200,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/200
100/100 [==============================] - 24s 224ms/step - loss: 0.3509 - val_loss: 0.4312
Epoch 2/200
100/100 [==============================] - 23s 230ms/step - loss: 0.1736 - val_loss: 0.3324
Epoch 3/200
100/100 [==============================] - 24s 245ms/step - loss: 0.1374 - val_loss: 0.1013
Epoch 4/200
100/100 [==============================] - 23s 229ms/step - loss: 0.1043 - val_loss: 0.2066
Epoch 5/200
100/100 [==============================] - 22s 226ms/step - loss: 0.0941 - val_loss: 0.0536
Epoch 6/200
100/100 [==============================] - 22s 221ms/step - loss: 0.0862 - val_loss: 0.0463
Epoch 7/200
100/100 [==============================] - 22s 224ms/step - loss: 0.0808 - val_loss: 0.0484
Epoch 8/200
100/100 [==============================] - 22s 223ms/step - loss: 0.0802 - val_loss: 0.0425
Epoch 9/200
100/100 [==============================] - 24s 240ms/step - loss: 0.0767 - val_loss: 0.0811
Epoch 10/200
100/100 [==============================] - 22s 224m

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

511/511 [==============================] - 20s 38ms/step - loss: 0.0401


In [ ]:
model.save("drive/MyDrive/Engineer's Project/longer_lstm_eur_usd.h5")

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.GRU(140,
  activation='sigmoid',
  #dropout=0.1,
  #recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(120,
  activation='sigmoid',))
model.add(layers.GRU(2,
  activation='sigmoid',
  #dropout=0.1,
  #recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=100,
  epochs=200,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/200
100/100 [==============================] - 34s 294ms/step - loss: 0.2515 - val_loss: 0.4053
Epoch 2/200
100/100 [==============================] - 29s 291ms/step - loss: 0.1533 - val_loss: 0.1123
Epoch 3/200
100/100 [==============================] - 31s 317ms/step - loss: 0.1120 - val_loss: 0.0318
Epoch 4/200
100/100 [==============================] - 29s 295ms/step - loss: 0.0940 - val_loss: 0.0747
Epoch 5/200
100/100 [==============================] - 28s 281ms/step - loss: 0.0844 - val_loss: 0.0586
Epoch 6/200
100/100 [==============================] - 28s 279ms/step - loss: 0.0783 - val_loss: 0.0238
Epoch 7/200
100/100 [==============================] - 28s 282ms/step - loss: 0.0748 - val_loss: 0.0223
Epoch 8/200
100/100 [==============================] - 28s 281ms/step - loss: 0.0750 - val_loss: 0.0408
Epoch 9/200
100/100 [==============================] - 30s 302ms/step - loss: 0.0731 - val_loss: 0.0599
Epoch 10/200
100/100 [==============================] - 27s 276m

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

511/511 [==============================] - 22s 44ms/step - loss: 0.0468


In [ ]:
model.save("drive/MyDrive/Engineer's Project/longer_gru_eur_usd.h5")

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(120,
  activation='sigmoid',
  #dropout=0.1,
  #recurrent_dropout=0.1,
  return_sequences=True,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.GRU(5,
  activation='sigmoid',
  #dropout=0.1,
  #recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(160,
  activation='sigmoid'))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(train_gen,
  steps_per_epoch=100,
  epochs=200,
  validation_data=val_gen,
  validation_steps=val_steps)

Epoch 1/200
100/100 [==============================] - 29s 262ms/step - loss: 0.1978 - val_loss: 0.4975
Epoch 2/200
100/100 [==============================] - 28s 281ms/step - loss: 0.1932 - val_loss: 0.3415
Epoch 3/200
100/100 [==============================] - 26s 262ms/step - loss: 0.1839 - val_loss: 0.0594
Epoch 4/200
100/100 [==============================] - 26s 260ms/step - loss: 0.1892 - val_loss: 0.0941
Epoch 5/200
100/100 [==============================] - 26s 265ms/step - loss: 0.2014 - val_loss: 0.3471
Epoch 6/200
100/100 [==============================] - 26s 260ms/step - loss: 0.1782 - val_loss: 0.1163
Epoch 7/200
100/100 [==============================] - 26s 264ms/step - loss: 0.1500 - val_loss: 0.0379
Epoch 8/200
100/100 [==============================] - 28s 282ms/step - loss: 0.1430 - val_loss: 0.1992
Epoch 9/200
100/100 [==============================] - 27s 268ms/step - loss: 0.1276 - val_loss: 0.2173
Epoch 10/200
100/100 [==============================] - 26s 262m

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

511/511 [==============================] - 23s 45ms/step - loss: 0.0472


In [ ]:
model.save("drive/MyDrive/Engineer's Project/longer_hybrid_eur_usd.h5")